In [ ]:
import pandas as pd
from datasets import dataset_dict, load_dataset,Dataset,DatasetDict
from tqdm import tqdm
import random
import string
import re

일반상식문장생성데이터 -> 단어 단위로 분류해서 집합  
논문자료요약 -> 기존에서 절반은 요약(0번) 가져오기 -> 나머지 절반은 단어 단위로 분류해서 집합으로

In [ ]:
new_df=pd.read_parquet('./13/논문자료요약.parquet')

In [ ]:
filtered_df = new_df#[new_df['domain'].isin(domains)]


In [ ]:
filtered_df

In [ ]:
for i in range(len(filtered_df)):
    
    filtered_df['text'][i] = filtered_df['text'][i].strip()
    
    try:
        if filtered_df['text'][i].endswith('. '):
            filtered_df['text'][i]=filtered_df['text'][i][:-1]

        elif filtered_df['text'][i].endswith('.'):
            pass

        elif filtered_df['text'][i].endswith(' .'):
            print(i, '번째 문장')
            print(filtered_df['text'][i])
            filtered_df['text'][i]=filtered_df['text'][i][:-2]+'.'

            print(filtered_df['text'][i])
            

        else:
            print(i, '번째 문장')
            print(filtered_df['text'][i])


            if filtered_df['text'][i].endswith(' '):
                filtered_df['text'][i]=filtered_df['text'][i][:-1]+'.'
                

            else:
                filtered_df['text'][i]=filtered_df['text'][i]+'.'
                
            
            print(filtered_df['text'][i])


    except Exception as e:
        print(e)
        print(i)
        print(filtered_df['text'][i])

        print('######################################################')

In [ ]:
remove_index = []
for i in range(len(filtered_df)):
    text = filtered_df['text'][i]
    
    if '�' in text:
        remove_index.append(i)
    elif '삭제.' in text[-5:]:
        remove_index.append(i)

In [ ]:
print(filtered_df['text'][remove_index[6]])

In [ ]:
filtered_df.drop(remove_index, inplace=True)
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df= filtered_df.dropna(subset=['text'])


In [ ]:
filtered_df=filtered_df.reset_index(drop=True)

In [ ]:
def cal_element(text):
    return len(text.split('. '))

In [ ]:
filtered_df['len_elements'] = filtered_df['text'].apply(lambda x: len(x.split('. ')) if isinstance(x, str) else 1)


In [ ]:
filtered_df

In [ ]:
filtered_df_1 = filtered_df[(filtered_df['len_elements'] > 2)&(filtered_df['len_elements'] < 15)] # 15기준
filtered_df_2 = filtered_df[filtered_df['len_elements'] <= 2]

In [ ]:
filtered_df_1=filtered_df_1.reset_index(drop=True)

In [ ]:
filtered_df_1

In [ ]:
filtered_df_2=filtered_df_2.reset_index(drop=True)

In [ ]:
filtered_df_2

In [ ]:
def make_text_allign_data(text):

    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'



    if text.endswith('.'):
        text+=' '



    my_list = text.split('. ')
    my_list_with_dots = [element + '.' for element in my_list]

    if my_list_with_dots[-1]=='.':

        my_list_with_dots=my_list_with_dots[:-1]



    random.shuffle(my_list_with_dots)


    if len(my_list_with_dots)==1:
        raise Exception("allign 리스트의 길이가 1입니다.",text)


    tmp_input='당신은 인공지능 비서입니다. 주어진 문장 리스트를 모두 활용하여 가장 정확한 단락을 생성하세요.'
    tmp_inst=f'이 문장들은 임의의 순서로 섞여 있습니다. 모든 문장을 활용하여 원본 단락의 순서와 내용을 올바른 순서로 재구성하세요.\n#문장 리스트: {my_list_with_dots}'
    
    
    if text.endswith(' '):
        tmp_out=text[:-1]

    else:
        tmp_out=text

        

    return tmp_input, tmp_inst, tmp_out



    






In [ ]:
def make_completion_data(text):


    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'



    if text.endswith('. '):
        text=text

    else:
        text=text+' '



    my_list_with_dots = text.split('. ')


#    print(my_list_with_dots)


    # ㅁㅁㅁㅁㅁ. 이렇게 공백있이 끝난경우
    if my_list_with_dots[-1]=='':

        my_list_with_dots=my_list_with_dots[:-1]

        if len(my_list_with_dots)==1:
            raise Exception("Completion 텍스트를 나눈  리스트의 길이가 1입니다.",text)

        last_sentence=my_list_with_dots.pop()
        last_sentence+='.'


        
    remaining_paragraph=''




    for i in range(len(my_list_with_dots)):
        
        if i!=len(my_list_with_dots)-1:
            remaining_paragraph+=my_list_with_dots[i]+'. '

        else:
            remaining_paragraph+=my_list_with_dots[i]+'.'





    tmp_input='당신은 인공지능 비서입니다. 주어진 원문을 바탕으로 주어진 질문에 가장 적절한 답변을 생성하세요.'

    tmp_instruct=f'다음 텍스트에서 제공된 문맥을 정확히 이해하고, 마지막 문장을 자연스럽고 문맥에 맞게 완성하세요. 문장은 이전 내용과 논리적으로 연결되어야 합니다.\n#텍스트: {remaining_paragraph}'

    tmp_output=last_sentence


    return tmp_input, tmp_instruct, tmp_output
    


    

    

    
    

In [ ]:
def make_text_mask_data(text):


    if not (text.endswith('.') or text.endswith('. ')):
        #print(text)

        text+='.'

    if text.endswith('. '):
        text=text[:-1]

    else:
        text=text


    words = re.findall(r'[가-힣]{2,}', text)
    
    random_word = random.choice(words)

    

  

    masked_text = text.replace(random_word, "<MASK>")


    tmp_input='당신은 인공지능 비서입니다. 주어진 질문에 가장 적절한 답변을 제공하세요.'
    tmp_instruct=f'이 문제에서는 주어진 텍스트 내의 <MASK>로 표시된 부분에 들어갈 적절한 단어를 예측해야 합니다. <MASK>가 위치한 문장의 전체 문맥을 분석하여, 문장의 나머지 내용과 일관되게 <MASK>에 들어갈 가장 적합한 단어를 답하세요.\n#텍스트: {masked_text}'
    tmp_output=random_word



    return tmp_input, tmp_instruct, tmp_output






In [ ]:
def make_word_align(text):

    word_lst=[]
    for word in text.split(' '):
        out = re.sub(r"[^\w\s]", "", word)
        word_lst.append(out)


    # 중복 제거하기 위해 set으로 만듬
    word_lst=set(word_lst)


    # 다시 리스트 형식으로 돌림
    word_lst=list(word_lst)


    # 랜덤하게 재배열
    random.shuffle(word_lst)


    tmp_input= '당신은 인공지능 비서입니다. 주어진 지시사항에 따라 가장 적절한 문장을 생성하세요.'
    tmp_instruct=f'이 문제에는 문장에서 공백을 기준으로 나누고, 구두점을 제거한 무작위로 섞인 단어들이 담긴 리스트가 제공됩니다. 이 리스트의 단어를 모두 활용하여 가장 문맥상 적절한 문장을 생성하세요.\n#단어리스트: {word_lst}'
    tmp_output=text

    return tmp_input, tmp_instruct, tmp_output










In [ ]:
input_lst=[]
output_lst=[]
inst_lst=[]
id_lst=[]


for i in range(len(filtered_df_1)):
    try:

        text=filtered_df_1['text'][i]

        

        thred=random.random()



        if thred < 0.44 :

            tmp_input, tmp_instruct, tmp_output = make_completion_data(text)
            tmp_id='completion_aihub'

        elif thred < 0.88:
            tmp_input, tmp_instruct, tmp_output = make_text_allign_data(text)
            tmp_id='text_allign_aihub'
      


        else:
            
            tmp_input, tmp_instruct, tmp_output = make_text_mask_data(text)
            tmp_id='pre_mask_aihub'




        input_lst.append(tmp_input)
        inst_lst.append(tmp_instruct)
        output_lst.append(tmp_output)
        id_lst.append(tmp_id)










    except Exception as e:
        
        print(e)
        print(filtered_df_1['text'][i])
        print('what_the Fuck?')
    
            

In [ ]:
len(filtered_df_1)

In [ ]:
len(input_lst)

In [ ]:
len(inst_lst)



In [ ]:
len(output_lst)



In [ ]:
len(id_lst)

In [ ]:
hub_df_1=pd.DataFrame({'input':input_lst,'instruction':inst_lst,'output':output_lst})

In [ ]:
input_lst=[]
output_lst=[]
inst_lst=[]
id_lst=[]


for i in range(len(filtered_df_2)):
    try:

        text=filtered_df_2['text'][i]

        

        thred=random.random()

        if thred < 0.56 :

            tmp_input, tmp_instruct, tmp_output = make_word_align(text)
            tmp_id='word_allign_aihub'

        else:
            
            tmp_input, tmp_instruct, tmp_output = make_text_mask_data(text)
            tmp_id='pre_mask_aihub'




        input_lst.append(tmp_input)
        inst_lst.append(tmp_instruct)
        output_lst.append(tmp_output)
        id_lst.append(tmp_id)



    except Exception as e:
        
        print(e)
        print(filtered_df_2['text'][i])
        print('what_the Fuck?')
    
            

In [ ]:
len(filtered_df_2)

In [ ]:
len(output_lst)


In [ ]:
hub_df_2=pd.DataFrame({'input':input_lst,'instruction':inst_lst,'output':output_lst})

In [ ]:
#hub_df=pd.concat([hub_df_1,hub_df_2],axis=0)
hub_df = hub_df_2.copy()

In [ ]:
hub_df=hub_df.reset_index(drop=True)

In [ ]:
hub_df

In [ ]:
hub_df.iloc[0,2]

In [ ]:
total_df=pd.concat([concat_df,hub_df],axis=0)

In [ ]:
hub_df.to_parquet('./Save/논문자료요약.parquet', engine = 'pyarrow', index = False)